In [9]:
import sys
import os
import pickle as pkl
import pandas as pd

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

from src.protein_graph import pncaGraph

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%aimport src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_seqs = pd.read_csv('../data/real_train_sequences.csv')
test_seqs = pd.read_csv('../data/real_test_sequences.csv')

In [3]:
pnca = pncaGraph(pdb='../pdb/3PL1-PZA.pdb',
# pnca = pncaGraph(pdb='../pdb/test_structures/0_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
                    lig_resname='PZA', 
                    self_loops=False,
                    # lig_pdb='../pdb/3PL1-PZA.pdb',
                    cutoff_distance=8)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [4]:
pnca_m = pncaGraph(pdb='../pdb/train_structures/0_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb',
                    lig_resname='PZA', 
                    self_loops=False,   
                    cutoff_distance=8)

In [5]:
test_structs = '../pdb/test_structures'

def create_graphs(structs_path, seqs):
    
    output_dict = {}
    
    for f in tqdm(os.listdir(structs_path)):
        
        index = f[:f.find('_')]
        name = 'pnca_mut_' + index

        pnca_m = pncaGraph(pdb=f'{structs_path}/{f}',
                        lig_resname='PZA', 
                        self_loops=False,
                        lig_pdb='../pdb/3PL1-PZA.pdb',
                        cutoff_distance=8)
        
        metadata = seqs.iloc[[int(index)]]
        # display(metadata)

        output_dict[name] = {
            'graph':pnca_m, 
            'metadata':metadata
            }
        
    return output_dict

In [6]:
test_structs = '../pdb/test_structures'
train_structs = '../pdb/train_structures'
train_graph_dict = create_graphs(train_structs, train_seqs)
test_graph_dict = create_graphs(test_structs, test_seqs)

100%|██████████| 200/200 [00:05<00:00, 36.55it/s]


In [7]:
# structs = '../pdb/test_structures'
# test_graph_dict = {}

# for f in tqdm(os.listdir(structs)):
    
#     index = f[:f.find('_')]
#     name = 'pnca_mut_' + index
#     f_path = f''
#     pnca_m = pncaGraph(pdb=f'../pdb/test_structures/{f}',
#                     lig_resname='PZA', 
#                     self_loops=False,
#                     lig_pdb='../pdb/3PL1-PZA.pdb',
#                     cutoff_distance=8)
    
#     metadata = test_seqs.iloc[[int(index)]]

#     test_graph_dict[name] = {
#         'graph':pnca_m, 
#         'metadata':metadata
#         }

In [12]:
wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'
# attach sequence from fast to each key in dictionary
# iterate through dict and creat one sample with gen_dataset per graph
# assign dataset object to dictionary
# will need to fix how to do CLAV dist

#! for now: assume coordinates of CLAV is the same as WT
#! use VMD to map CLAV onto mutants?

for sample in tqdm(test_graph_dict):
        
    test_graph_dict[sample]['graph'].gen_dataset(
        wt_seq=wt_seq,
        sequences= test_graph_dict[sample]['metadata'],
        edge_weights= 'none',
        normalise=True
        )
    
for sample in tqdm(train_graph_dict):
    
    train_graph_dict[sample]['graph'].gen_dataset(
        wt_seq=wt_seq,
        sequences= train_graph_dict[sample]['metadata'],
        edge_weights= 'none',
        normalise=True
        )

100%|██████████| 464/464 [01:42<00:00,  4.53it/s]


In [ ]:
# TODO: append Data objects togethe as lists, then output datasets as .pths

In [51]:
for sample in test_graph_dict:
    print(test_graph_dict[sample]['graph'].dataset[0])

Data(x=[185, 16], edge_index=[2, 1840], y=1)
Data(x=[185, 16], edge_index=[2, 1838], y=0)
Data(x=[185, 16], edge_index=[2, 1838], y=1)
Data(x=[185, 16], edge_index=[2, 1830], y=0)
Data(x=[185, 16], edge_index=[2, 1844], y=0)
Data(x=[185, 16], edge_index=[2, 1834], y=0)
Data(x=[185, 16], edge_index=[2, 1828], y=1)
Data(x=[185, 16], edge_index=[2, 1836], y=1)
Data(x=[185, 16], edge_index=[2, 1832], y=0)
Data(x=[185, 16], edge_index=[2, 1828], y=1)
Data(x=[185, 16], edge_index=[2, 1838], y=0)
Data(x=[185, 16], edge_index=[2, 1834], y=0)
Data(x=[185, 16], edge_index=[2, 1844], y=0)
Data(x=[185, 16], edge_index=[2, 1830], y=0)
Data(x=[185, 16], edge_index=[2, 1832], y=1)
Data(x=[185, 16], edge_index=[2, 1836], y=1)
Data(x=[185, 16], edge_index=[2, 1840], y=1)
Data(x=[185, 16], edge_index=[2, 1838], y=1)
Data(x=[185, 16], edge_index=[2, 1836], y=0)
Data(x=[185, 16], edge_index=[2, 1836], y=1)
Data(x=[185, 16], edge_index=[2, 1846], y=0)
Data(x=[185, 16], edge_index=[2, 1840], y=0)
Data(x=[18